In [88]:
import numpy as np
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

# P1 data
data1_path = '../Datasets/TaxiBJ/TAXIBJ2013.grid'
# P2 data
data2_path = '../Datasets/TaxiBJ/TAXIBJ2014.grid'
# P3 data
data3_path = '../Datasets/TaxiBJ/TAXIBJ2015.grid'
# P4 data
data4_path = '../Datasets/TaxiBJ/TAXIBJ2016.grid'
# external data
extdatapath = '../Datasets/TaxiBJ/TAXIBJ.ext'

# load P1 data
xdata1 = pd.read_csv(data1_path)
xdata1.head()
# load P2 data
xdata2 = pd.read_csv(data2_path)
# load P3 data
xdata3 = pd.read_csv(data3_path)
# load P4 data
xdata4 = pd.read_csv(data4_path)
# load external data
extdata = pd.read_csv(extdatapath)
extdata.head()
extdata.shape

,dyna_id,type,time,row_id,column_id,inflow,outflow
0,0,state,2013-07-01T00:00:00Z,0,0,0.0,0.0
1,1,state,2013-07-01T00:30:00Z,0,0,0.0,0.0
2,2,state,2013-07-01T01:00:00Z,0,0,0.0,0.0
3,3,state,2013-07-01T01:30:00Z,0,0,0.0,0.0
4,4,state,2013-07-01T02:00:00Z,0,0,0.0,0.0


,ext_id,time,Temperature,WindSpeed,Weather0,Weather1,Weather2,Weather3,Weather4,Weather5,...,Weather7,Weather8,Weather9,Weather10,Weather11,Weather12,Weather13,Weather14,Weather15,Weather16
0,0,2013-02-01T02:00:00Z,-2.6,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2013-02-01T02:30:00Z,-2.6,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2013-02-01T03:00:00Z,-2.6,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2013-02-01T03:30:00Z,-2.6,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2013-02-01T04:00:00Z,-2.6,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(59006, 21)

In [89]:
#xdata1 = xdata1.drop(columns=['dyna_id', 'type'])
xdata1_gb = xdata1.groupby('time')#.apply(print)
xdata2_gb = xdata2.groupby('time')
xdata3_gb = xdata3.groupby('time')#.apply(print)
xdata4_gb = xdata4.groupby('time')

# number of groups
xdata1_gb.ngroups
xdata2_gb.ngroups
xdata3_gb.ngroups
xdata4_gb.ngroups

# convert to list
xdata1_gb = list(xdata1_gb)
xdata2_gb = list(xdata2_gb)
xdata3_gb = list(xdata3_gb)
xdata4_gb = list(xdata4_gb)

4848

4368

5520

6624

In [90]:
# nth data (nth, 2, 32, 32)
n = 0
# xdata1_gb[n][0] is the specific 30 minute interval time
data = xdata1_gb[n][1] # the associated dataframe
data_inflow = data.pivot_table(values='inflow', index='row_id', columns='column_id').to_numpy()
data_outflow = data.pivot_table(values='outflow', index='row_id', columns='column_id').to_numpy()

data_inflow
data_outflow

array([[  0.,  20.,  41., ...,   7.,   1.,  40.],
       [  0.,  11.,   8., ...,  22.,   7.,  96.],
       [  0.,   9.,   5., ...,  14., 198., 117.],
       ...,
       [  1.,   3.,   4., ...,   8.,   6.,   3.],
       [  0.,   0.,   0., ...,   9.,   0.,   7.],
       [  1.,   0.,   0., ...,  17.,  20.,   7.]])

array([[  0.,  21.,  44., ...,   6.,   1.,  38.],
       [  0.,  11.,   8., ...,  23.,   6.,  93.],
       [  0.,   9.,   5., ...,  15., 201., 120.],
       ...,
       [  2.,   3.,   4., ...,   7.,   7.,   3.],
       [  0.,   0.,   0., ...,   9.,   0.,   7.],
       [  1.,   0.,   0., ...,  19.,  21.,   8.]])

In [92]:
# convert data to numpy array and save as file
for j in range(1,2):
    size = len(eval(f'xdata{j}_gb'))
    flow_data = np.empty([size, 2, 32, 32])
    ext_data = np.empty([size, 27]) # 7 day of week, holiday, temp, windspeed, 17 weathers
    for i in range(size):
        # flow data
        data = eval(f'xdata{j}_gb[i][1]')
        data_inflow = data.pivot_table(values='inflow', index='row_id', columns='column_id').to_numpy()
        data_outflow = data.pivot_table(values='outflow', index='row_id', columns='column_id').to_numpy()
        flow_data[i, 0] = data_inflow
        flow_data[i, 1] = data_outflow
        # external data
        time = eval(f'xdata{j}_gb[i][0]')
        date = pd.to_datetime(time, format='%Y-%m-%dT%H:%M:%SZ').dayofweek
        days = np.zeros([1, 8])
        days[0, date] = 1
        if date > 4: # is a weekend date
            days[0, 7] = 1 # holiday index
        edata = extdata.loc[extdata['time']==time].drop(columns=['ext_id', 'time']).drop_duplicates()
        ext_data[i] = np.concatenate((days, edata), axis=1)
    np.save(f'../Datasets/TaxiBJ{2012+j}', flow_data)
    np.save(f'../Datasets/TaxiBJext{2012+j}', ext_data)


In [11]:
# split TaxiBJ2016 in half since file exceeds 100MB
data = np.load('../Datasets/TaxiBJ2016.npy')
half = int(len(data)/2)
data1 = data[0:half]
data2 = data[half:]
np.save(f'../Datasets/TaxiBJ2016_1', data1)
np.save(f'../Datasets/TaxiBJ2016_2', data2)

In [83]:
# verify sizes
data = np.load('../Datasets/TaxiBJ2013.npy')
data2 = np.load('../Datasets/TaxiBJext2013.npy')
data.shape
data2.shape

(4848, 2, 32, 32)

(4848, 27)